## Agent to interact with CRM system
- 1.0 Database is created using SQL and populated with dummy data
- 2.0 Functions to add, amend, and query records is added to tool
- 3.0 Wrap functions as ADK tool Function
- 4.0 attach tools to the agent

In [29]:
# Load the SQL extension
import sqlite3
from sqlite3 import Error
# Load the SQL extension
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection


In [ ]:
# Create a connection to the database
connection = create_connection('crmhotel.db')
# print(f"\n✅ connection to {connection} successful")

Connection to SQLite DB successful

✅ connection to <sqlite3.Connection object at 0x73cb10f67010> successful


In [ ]:
# Create tables. Run this cell once only. Once the tables are created the database is ready.
# Customers Table

create_customers_table = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    company_name VARCHAR(100),
    email VARCHAR(100) UNIQUE,
    phone VARCHAR(20),
    address TEXT,
    city VARCHAR(50),
    state VARCHAR(50),
    zip_code VARCHAR(10),
    industry VARCHAR(50),
    tier VARCHAR(10),
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""

# contacts table
create_contacts_table ="""
CREATE TABLE IF NOT EXISTS contacts (
    contact_id INT PRIMARY KEY,
    customer_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone VARCHAR(20),
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""
# Opportunities Table
create_opportunities_table = """
CREATE TABLE IF NOT EXISTS opportunities (
    opportunity_id INT PRIMARY KEY,
    customer_id INT,
    contact_id INT,
    description TEXT,
    value DECIMAL(10, 2),
    status VARCHAR(50),
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (contact_id) REFERENCES contacts(contact_id)
);
"""

# Activities Table
create_activities_table = """
CREATE TABLE IF NOT EXISTS activities (
    activity_id INT PRIMARY KEY,
    opportunity_id INT,
    customer_id INT,
    contact_id INT,
    activity_type VARCHAR(50),
    subject VARCHAR(100),
    description TEXT,
    activity_date TIMESTAMP,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (contact_id) REFERENCES Contacts(contact_id),
    FOREIGN KEY (opportunity_id) REFERENCES opportunities(opportunity_id)           
);
"""

# Users Table
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    user_id INT PRIMARY KEY,
    username VARCHAR(50) UNIQUE,
    password_hash VARCHAR(255),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100) UNIQUE,
    role VARCHAR(50),
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""

# Trigger to update 'updated_at' field on row update
create_update_timestamp_trigger = """
CREATE TRIGGER IF NOT EXISTS update_timestamp
AFTER UPDATE ON customers
FOR EACH ROW
BEGIN
    UPDATE customers SET updated_at = CURRENT_TIMESTAMP WHERE customer_id = OLD.customer_id;
END;
"""
# Trigger to update 'updated_at' field on row update for contacts table
create_update_contacts_timestamp_trigger = """
CREATE TRIGGER IF NOT EXISTS update_contacts_timestamp
AFTER UPDATE ON contacts
FOR EACH ROW
BEGIN
    UPDATE contacts SET updated_at = CURRENT_TIMESTAMP WHERE contact_id = OLD.contact_id;
END;
"""

# Trigger to update 'updated_at' field on row update for opportunities table
create_update_opportunities_timestamp_trigger = """
CREATE TRIGGER IF NOT EXISTS update_opportunities_timestamp
AFTER UPDATE ON opportunities
FOR EACH ROW
BEGIN
    UPDATE opportunities SET updated_at = CURRENT_TIMESTAMP WHERE opportunity_id = OLD.opportunity_id;
END;
""" 
# Trigger to update 'updated_at' field on row update for activities table
create_update_activities_timestamp_trigger = """
CREATE TRIGGER IF NOT EXISTS update_activities_timestamp
AFTER UPDATE ON activities
FOR EACH ROW
BEGIN
    UPDATE activities SET updated_at = CURRENT_TIMESTAMP WHERE activity_id = OLD.activity_id;
END;
""" 
# Trigger to update 'updated_at' field on row update for users table
create_update_users_timestamp_trigger = """
CREATE TRIGGER IF NOT EXISTS update_users_timestamp
AFTER UPDATE ON users
FOR EACH ROW
BEGIN
    UPDATE users SET updated_at = CURRENT_TIMESTAMP WHERE user_id = OLD.user_id;
END;
""" 


In [31]:
# Execute SQL queries
def execute_query(connection, query):
    cursor = connection.cursor()          
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Exception as e:
        print(f"An error occurred: {e}")
        return cursor.fetchall()

In [ ]:
# use execute_query function to create tables
execute_query(connection, create_customers_table)
execute_query(connection, create_contacts_table)
execute_query(connection, create_opportunities_table)
execute_query(connection, create_activities_table)
execute_query(connection, create_users_table)

In [ ]:
# Create triggers to update 'updated_at' fields
execute_query(connection, create_update_timestamp_trigger)
execute_query(connection, create_update_contacts_timestamp_trigger)
execute_query(connection, create_update_opportunities_timestamp_trigger)
execute_query(connection, create_update_activities_timestamp_trigger)
execute_query(connection, create_update_users_timestamp_trigger)    

In [32]:
# List database tables - Helper
def list_tables() -> list[str]:
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])
    cursor.close()
    return [table[0] for table in tables]
tables = list_tables()

Tables in the database:
customers
contacts
opportunities
activities
users


In [33]:
# List database triggers - Helper
def list_triggers() -> list[str]:
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='trigger';")
    triggers = cursor.fetchall()
    print("Triggers in the database:")
    for trigger in triggers:
        print(trigger[0])
    cursor.close()
    return [trigger[0] for trigger in triggers]
triggers = list_triggers()

Triggers in the database:
update_timestamp
update_contacts_timestamp
update_opportunities_timestamp
update_activities_timestamp
update_users_timestamp


In [ ]:
%pip install -qU google-adk
from google.adk.agents import Agent
from google.adk.agents import LlmAgent

In [ ]:
 # setting up the google AI studio API
%pip install -qU dotenv
import os
from dotenv import load_dotenv
# from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

## Agent defined tools
 - Function to query customer table
 - Function to update activity table

In [ ]:
""" # Query customer table function
def query_customers(db_path: str, query: str) -> list:
    """
    Retrieves the customer ID, status, and name from the customers table.
    Args:
        db_path (str): Path to the SQLite database file.
        query (str): SQL SELECT query to execute.
    Returns:
        list: List of tuples containing the query results.  
    """
    connection = create_connection(db_path)
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    finally:
        cursor.close()
        connection.close()

def insert_data(db_path: str, query: str, data: dict) -> bool:
    """
    Insert a row in the activity table based on the type of customer's request.
    args:
        db_path (str): Path to the SQLite database file.
        query (str): SQL INSERT query to execute.
        data (dict): A dictionary containing the data to be inserted.
    returns:
        bool: True if the insertion was successful, False otherwise.
    """
    connection = create_connection(db_path)
    cursor = connection.cursor()
    columns = ', '.join(data.keys())
    placeholders = ', '.join(['?' for _ in data])
    query = f"INSERT INTO table_name ({columns}) VALUES ({placeholders})"
    try:
        cursor.execute(query, tuple(data.values()))
        connection.commit()
        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    finally:
        cursor.close() """

## Agent portion
 - Function to query customer table
 - Function to update activity table

In [ ]:
""" # Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.") """

In [ ]:
# Query customer table function
def query_customers(db_path: str, query: str, data: dict) -> list:
    """
    Retrieves the customer ID, status, and name from the customers table.
    Args:
        db_path (str): Path to the SQLite database file.
        query (str): SQL SELECT query to execute.
    Returns:
        list: List of tuples containing the query results.  
    """
    connection = create_connection(db_path)
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    finally:
        cursor.close()
        connection.close()

def insert_data(db_path: str, query: str, data: dict) -> bool:
    """
    Insert a row in the activity table based on the type of customer's request.

    args:
        db_path (str): Path to the SQLite database file.
        query (str): SQL INSERT query to execute.
        data (dict): A dictionary containing the data to be inserted.
    
    returns:
        bool: True if the insertion was successful, False otherwise.
    """
    connection = create_connection(db_path)
    cursor = connection.cursor()
    columns = ', '.join(data.keys())
    placeholders = ', '.join(['?' for _ in data])
    query = f"INSERT INTO table_name ({columns}) VALUES ({placeholders})"
    try:
        cursor.execute(query, tuple(data.values()))
        connection.commit()
        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    finally:
        cursor.close()
        connection.close()


In [ ]:
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner
from typing import Any
from typing import Dict
from google.adk.apps.app import App
from google.adk.apps.app import EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.tools.tool_context import ToolContext
print("\n✅ Google ADK setup completed.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

🧭 Triage Agent

- Define the Enterprise CRM Agent 🛠️ 

The Triage Agent needs to retrieve the customer's details and check it's status first. This is implemented using using custom Agent querying an SQlite database. 

In [ ]:
""" crmagent = Agent(
    name = "CRMManagerAgent",
    model = Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description = "Intelligent CRM agent",
    instruction =
        """  
        You are an expert in customer relationship management (CRM) systems.
        Steps to follow:
        1.0 **Use** the `query_customers` tool to fetch customer details when needed.
        2.0 **Use** the `insert_data` tool to log new customer requests or activities into the CRM database.
        3.0 Always ensure data integrity and accuracy when performing database operations.
        4.0 Respond with appropriate confirmations or data based on the operations performed.
        5.0 If you encounter any issues or errors during database operations, provide clear error messages.
        6.0 Maintain a professional and helpful tone in all interactions.
        7.0 Prioritize customer satisfaction and efficient database management in all tasks.
        8.0 Adhere to best practices for database management and customer service throughout your operations
        """,
)                   
output_key="crm_response"
print("\n✅ Message Triage Agent successfully created and integrated with the custom Agents.")
 """

In [ ]:
""" memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing
# 1. Define required constants
USER_ID = "crm_agent" 
# Use a dedicated name for the App (best practice)
APP_NAME = "crm_agentdb_app" 
MODEL_NAME = "ReplyAgent"

session_service = InMemorySessionService()

runner = Runner(
    agent = crmagent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,  
)

print("✅ Agent and Runner created with memory support!") """

In [ ]:
""" # pass the runner instance (lowercase `runner`) instead of the Runner class
await run_session(
    runner,
    "My name isJoe, my email is user@unknow and customer ID cust-01. I am having an issue with my delivery.",
    "conversation-01",  # Session ID
)
 """

In [ ]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

In [ ]:
crm_agent = LlmAgent(
    name = "CRM_ManagerAgent",
    model = Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description = "Intelligent CRM agent with persistent memory",
    instruction =
        """  
        You are an expert in customer relationship management (CRM) systems.
        Steps to follow:
        1.0 **Use** the `query_customers` tool to fetch customer details when needed.
        2.0 **Use** the `insert_data` tool to log new customer requests or activities into the CRM database.
        3.0 Always ensure data integrity and accuracy when performing database operations.
        4.0 Respond with appropriate confirmations or data based on the operations performed.
        5.0 If you encounter any issues or errors during database operations, provide clear error messages.
        6.0 Maintain a professional and helpful tone in all interactions.
        7.0 Prioritize customer satisfaction and efficient database management in all tasks.
        8.0 Adhere to best practices for database management and customer service throughout your operations
        """,
)
# Implement DatabaseSessionService using SQlite database
db_url = 'sqlite+aiosqlite:///crm_agent_data.db'
session_service = DatabaseSessionService(db_url = db_url)                  
output_key="crm_response"
print("\n✅ Message Triage Agent successfully created and integrated with the custom Agents.")


In [ ]:
#Setup runner
USER_ID = "crm_agent" 
# Use a dedicated name for the App (best practice)
APP_NAME = "crm_agentdb_app" 
MODEL_NAME = "ReplyAgent"
runner = Runner(agent=crm_agent, app_name=APP_NAME, session_service=session_service)

In [ ]:
# pass the runner instance (lowercase `runner`) instead of the Runner class
await run_session(
    runner,
    ["My name isJoe, my email is user@unknow and customer ID cust-01. I am having an issue with my delivery."],
    "test-db-session-01",  # Session ID
)